In [12]:
import heapq  # For priority queues
import time   # For timing
import math   # For Haversine
import folium # For visualization
from folium.plugins import AntPath
import pandas as pd  # For matrix 
from IPython.display import display
import csv


In [13]:


with open('data/cities_of_srilanka2.csv', 'r') as f:
    reader = csv.DictReader(f)
    cities = []
    for row in reader:
        if any('truncated' in str(value) for value in row.values()):
            continue
        
        latitude = row.get('latitude')
        longitude = row.get('longitude')
        name = row.get('name_en')
        if name and latitude and longitude:
            try:
                lat = float(latitude)
                lon = float(longitude)
                if not math.isnan(lat) and not math.isnan(lon):
                    cities.append({'name': name.strip(), 'lat': lat, 'lon': lon})
            except ValueError:
                pass

# Remove duplicates
unique_cities = {}
for c in cities:
    if c['name'] not in unique_cities:
        unique_cities[c['name']] = c

city_list = list(unique_cities.values())
city_names = [c['name'] for c in city_list]
city_coords = [(c['lat'], c['lon']) for c in city_list]
N = len(city_names)

print(f"Loaded {N} unique cities.")

Loaded 1844 unique cities.


In [14]:
def haversine(coord1, coord2):
    R = 6371.0
    lat1, lon1 = math.radians(coord1[0]), math.radians(coord1[1])
    lat2, lon2 = math.radians(coord2[0]), math.radians(coord2[1])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

In [15]:
graph = {name: {} for name in city_names}
nearest_cities = {name: [] for name in city_names}

for i in range(N):
    city = city_names[i]
    distances = []
    for j in range(N):
        if i != j:
            dist = haversine(city_coords[i], city_coords[j])
            distances.append((dist, city_names[j]))
    
    # Sort and get 5 nearest
    distances.sort()
    nearest = distances[:5]
    nearest_cities[city] = [nb for _, nb in nearest]
    
    # Add edges (undirected)
    for dist, nb in nearest:
        graph[city][nb] = dist
        graph[nb][city] = dist  # Make undirected

#  Display nearest for a few cities
print("\nFive Nearest Cities for Sample Cities:")
for city in city_names[:5]:
    print(f"{city}: {nearest_cities[city]}")

# Distance matrix (too large for full display, show subset)
subset_names = city_names[:10]
subset_matrix = pd.DataFrame(index=subset_names, columns=subset_names)
for city in subset_names:
    for nb in graph[city]:
        if nb in subset_names:
            subset_matrix.at[city, nb] = round(graph[city][nb], 2)
print("\nSubset Distance Matrix (km):")
display(subset_matrix)


Five Nearest Cities for Sample Cities:
Akkaraipattu: ['Kannakipuram', 'Oluvil', 'Thambiluvil', 'Tirukovil', 'Wanagamuwa']
Ambagahawatta: ['Hulannuge', 'Lahugala', 'Dorakumbura', 'Padiyatalawa', 'Kolamanthalawa']
Ampara: ['Deegawapiya', 'Digamadulla Weeragoda', 'Paragahakele', 'Uhana', 'Pahalalanda']
Bakmitiyawa: ['Pannalagama', 'Wadinagala', 'Kandaudapanguwa', 'Siyambalanduwa', 'Buddama']
Deegawapiya: ['Ampara', 'Digamadulla Weeragoda', 'Paragahakele', 'Uhana', 'Pahalalanda']

Subset Distance Matrix (km):


,Akkaraipattu,Ambagahawatta,Ampara,Bakmitiyawa,Deegawapiya,Devalahinda,Digamadulla Weeragoda,Dorakumbura,Gonagolla,Hulannuge
Akkaraipattu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ambagahawatta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.07,NaN,0.0
Ampara,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN
Bakmitiyawa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Deegawapiya,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
Devalahinda,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Digamadulla Weeragoda,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN
Dorakumbura,NaN,5.07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.07
Gonagolla,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Hulannuge,NaN,0.0,NaN,NaN,NaN,NaN,NaN,5.07,NaN,NaN


In [16]:
def a_star(graph, start, goal):
    frontier = []
    heapq.heappush(frontier, (0, start))
    came_from = {start: None}
    g_score = {start: 0}
    steps = []
    
    while frontier:
        f_score, current = heapq.heappop(frontier)
        steps.append(f"Exploring {current} with f {f_score:.2f} km")
        
        if current == goal:
            path = []
            while current is not None:
                path.append(current)
                current = came_from[current]
            path.reverse()
            return path, g_score[goal], steps
        
        for neighbor, weight in graph.get(current, {}).items():
            tentative_g = g_score[current] + weight
            if neighbor not in g_score or tentative_g < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g
                h = haversine(city_coords[city_names.index(neighbor)], city_coords[city_names.index(goal)])
                heapq.heappush(frontier, (tentative_g + h, neighbor))
    
    return None, float('inf'), steps

In [17]:


print("Available cities (first 10):", city_names[:10], "... (total 161)")
start_city = input("Enter source city: ").strip()
end_city = input("Enter destination city: ").strip()

if start_city not in city_names or end_city not in city_names:
    print("Invalid city. Choose from list.")
else:
    # Display 5 nearest cities to end_city
    print(f"\nFive nearest to {end_city}: {nearest_cities[end_city]}")
    
    # Run A* search
    start_time = time.time()
    path, dist, steps = a_star(graph, start_city, end_city)
    exec_time = (time.time() - start_time) * 1000
    
    # Output results
    print("\n--- A* Search ---")
    if path:
        print("Path:", " -> ".join(path))
        print(f"Distance: {dist:.2f} km")
        print(f"Time: {exec_time:.2f} ms")
        print("Steps (first 5):")
        for step in steps[:5]:
            print(step)
        if len(steps) > 5:
            print("... (more)")
        
        # Create comparison DataFrame
        comp_df = pd.DataFrame({
            "Algorithm": ["A* Search"],
            "Distance (km)": [dist],
            "Time (ms)": [exec_time]
        })
        display(comp_df)
        
        # Visualize path on map
        m = folium.Map(location=[7.5, 80.5], zoom_start=7)
        # Add markers only for cities in the path to reduce clutter
        for city in path:
            coord = city_coords[city_names.index(city)]
            folium.Marker(coord, popup=city, icon=folium.Icon(color='blue')).add_to(m)
        # Add path
        path_coords = [city_coords[city_names.index(city)] for city in path]
        AntPath(path_coords).add_to(m)
        display(m)
    else:
        print("No path found (graph may not be connected).")

Available cities (first 10): ['Akkaraipattu', 'Ambagahawatta', 'Ampara', 'Bakmitiyawa', 'Deegawapiya', 'Devalahinda', 'Digamadulla Weeragoda', 'Dorakumbura', 'Gonagolla', 'Hulannuge'] ... (total 161)



Five nearest to Matara: ['Nadugala', 'Diyagaha', 'Palatuwa', 'Sultanagoda', 'Kamburugamuwa']

--- A* Search ---
Path: Jaffna -> Puthukudiyiruppu -> Mannar -> Eluwankulama -> Kalkudah -> Mampuri -> Mukkutoduwawa -> Kottantivu -> Mundel -> Battuluoya -> Rajakadaluwa -> Jayasiripura -> Chilaw -> Ambakandawila -> Toduwawa -> Marawila -> Katuneriya -> Wennappuwa -> Dalukana -> Waikkal -> Kochchikade -> Negombo -> Katunayake -> Raddolugama -> Opatha -> Makewita -> Bollete (WP) -> Ganemulla -> Kadawatha -> Naranwala -> Dekatana -> Dompe -> Dedigamuwa -> Batawala -> Padukka -> Kahawala -> Miwanapalana -> Poruwedanda -> Diwalakada -> Bulathsinhala -> Mahagama -> Pimbura -> Agalawatta -> Matugama -> Welipenna -> Paraigama -> Mahakalupahana -> Walallawita -> Amugoda -> Porawagama -> Unenwitiya -> Mapalagama -> Udalamatta -> Akmeemana -> Nakiyadeniya -> Yakkalamulla -> Karagoda -> Penetiyana -> Denipitiya -> Sultanagoda -> Matara
Distance: 474.18 km
Time: 101.83 ms
Steps (first 5):
Exploring Jaff

,Algorithm,Distance (km),Time (ms)
0,A* Search,474.175013,101.828337
